# **DELAY PREDICTION**
In this notebook several models are implemented and tested out for predicting the delay of a sbb connection (= sequence of stops in the sbb network). They all output a numeric value which is the delay in minutes for the inputted connection.import os
import pandas as pd
pd.set_option("display.max_columns", 50)
import matplotlib.pyplot as plt
import warnings
import plotly.express as px
import plotly.graph_objects as go
warnings.simplefilter(action='ignore', category=UserWarning)

In [2]:
import os
import pandas as pd
pd.set_option("display.max_columns", 50)
import matplotlib.pyplot as plt
import warnings
import plotly.express as px
import plotly.graph_objects as go
warnings.simplefilter(action='ignore', category=UserWarning)
from pyhive import hive

In [3]:
# Set python variables from environment variables
username = os.environ['USERNAME']
hive_host = os.environ['HIVE_SERVER2'].split(':')[0]
hive_port = os.environ['HIVE_SERVER2'].split(':')[1]

In [4]:
# create connection
conn = hive.connect(
    host=hive_host,
    port=hive_port,
    # auth="KERBEROS",
    # kerberos_service_name = "hive"
)

# create cursor
cur = conn.cursor()

print(f"your username is {username}")
print(f"you are connected to {hive_host}:{hive_port}")

your username is hanygeor
you are connected to iccluster044.iccluster.epfl.ch:10000


In [ ]:
query = f"""
    SELECT month AS month_year, LOWER(produkt_id) AS ttype, COUNT(*) AS stops
    FROM {username}.sbb_orc 
    WHERE betriebstag LIKE '__.__.2022' AND produkt_id IS NOT NULL AND produkt_id!=''
    GROUP BY LOWER(produkt_id), month
    ORDER BY LOWER(produkt_id) ASC, month ASC 
"""
data = pd.read_sql(query, conn)

# Cleaning and data preparation
Consider tasks and its assumptions:
   1. We only consider departure and arrival stops in a 15km radius of Zürich's train station, Zürich HB (8503000), (lat, lon) = (47.378177, 8.540192)
   2. We only consider journeys at reasonable hours of the day, and on a typical business day, and assuming a recent schedule. --> TODO think: does this mean we should train our model without weekend days (= typical business day), connections during the night (=reasonable hours of the day) and a recent schedule (which means what??)

In [ ]:
query = f"""
    DROP TABLE IF EXISTS {username}.sbb_zurich
"""
cur.execute(query)

In [9]:
# filter for stops in a 15km radius of ZH train station (lat, lon) = (47.378177, 8.540192)
query = f"""
    CREATE TABLE {username}.sbb_zurich
    AS 
        SELECT *
        FROM {username}.sbb_stops_orc
        WHERE (6371 * acos(cos(radians(STOP_LAT)) * cos(radians(47.378177))
                        * cos(radians(8.540192) - radians(STOP_LON))
                        + sin(radians(STOP_LAT)) * sin(radians(47.378177)))) <= 15
"""
cur.execute(query)

In [ ]:
# Unused
# filter for stops in a 15km radius of ZH train station (lat, lon) = (47.378177, 8.540192)
query_filter_stops_near_zh = f"""
    SELECT *
    FROM {username}.sbb_stops_orc
    WHERE (6371 * acos(cos(radians(STOP_LAT)) * cos(radians(47.378177))
                    * cos(radians(8.540192) - radians(STOP_LON))
                    + sin(radians(STOP_LAT)) * sin(radians(47.378177)))) <= 15
"""
stops_near_zh_central = pd.read_sql(query_filter_stops_near_zh, conn)
stops_near_zh_central

In [10]:
query = f"""
    SELECT * FROM {username}.sbb_zurich
"""
df_koukou = pd.read_sql(query, conn)

In [11]:
df_koukou.head()

,sbb_zurich.stop_id,sbb_zurich.stop_name,sbb_zurich.stop_lat,sbb_zurich.stop_lon,sbb_zurich.location_type,sbb_zurich.parent_station
0,176,Zimmerberg-Basistunnel,47.351677,8.521957,,
1,8500926,"Oetwil a.d.L., Schweizäcker",47.423626,8.403183,,
2,8502075,"Zürich Flughafen, Carterminal",47.451023,8.563729,,
3,8502186,Dietikon Stoffelbach,47.393326,8.398960,,Parent8502186
4,8502186:0,Dietikon Stoffelbach,47.393400,8.398943,,Parent8502186


## Statistical model
Delay in minutes of a connection A to B is regarded as random variable. In this model we find the average delay with alpha 5% certainty. 

In [ ]:
data

In [ ]:
# set parameters
start = 0
destination = 1

In [ ]:
# import data from hdfs using hive (gives an error)
query = f"""
    SELECT ANKUNFTSZET, AN_PROGNOSE, ABFAHRTSZEIT, AB_PROGNOSE
    FROM {username}.sbb_orc 
"""

data = pd.read_sql(query, conn)

In [4]:
# get stops 15km around ZH central station 
# since "We only consider departure and arrival stops in a 15km radius of Zürich's train station, Zürich HB (8503000), (lat, lon) = (47.378177, 8.540192)

In [12]:
# Step 1: create table of delays


query = f"""
    DROP TABLE IF EXISTS {username}.sbb_delays_ic_gen
"""
cur.execute(query)

query = f"""
    CREATE TABLE {username}.sbb_delays_ic_gen
    STORED AS ORC
    AS WITH Times AS(
        SELECT linien_id AS train, 
               unix_timestamp(ANKUNFTSZEIT, 'dd.MM.yyyy HH:mm') AS expected, 
               unix_timestamp(AN_PROGNOSE, 'dd.MM.yyyy HH:mm:ss') AS actual
        FROM {username}.sbb_orc AS t_o
        JOIN {username}.sbb_zurich AS t_z ON t_z.stop_name = t_o.HALTESTELLEN_NAME
    )
    SELECT train, IF(actual > expected, actual - expected, 0) AS delay
    FROM Times
"""
cur.execute(query)

In [ ]:
# Step 2: get the stats out for the median and 3rd (.75) quartile

query = f"""
    SELECT train,
           percentile_approx(delay, 0.5) AS second, 
           percentile_approx(delay, 0.75) AS third
    FROM {username}.sbb_delays_ic_gen
    GROUP BY train
    ORDER BY train
"""
df_delays_ic_gen = pd.read_sql(query, conn)

In [ ]:
df_delays_ic_gen.head()

In [ ]:
query = f"""
    SELECT * FROM {username}.sbb_delays_ic_gen
"""
df_koukou = pd.read_sql(query, conn)